# Context managers for transactions

Create a realistic example of a context manager.  This example will involve a `Connection` class which represents some sort of database connection, along with a `Transaction` class which manages transactions in the database.  Users of the system can creat `Connections` and then create `Transaction` objects to start transactions.  To commit or rollback transactions, users can call methods on the `Transaction` instances.

`Connection` class:

In [1]:
class Connection:
    def __init__(self):
        self.xid = 0

    def _start_transaction(self):
        print('starting transaction', self.xid)
        rslt = self.xid
        self.xid = self.xid + 1
        return rslt

    def _commit_transaction(self, xid):
        print('committing transaction', xid)

    def _rollback_transaction(self, xid):
        print('rolling back transaction', xid)

Note that the above does not do _real_ database work.  All that it really does is increment a transaction-id whenever a new transaction is started.  It also prints out what it is doing as transactions are processed.  Now for the `Transaction` class:

In [2]:
class Transaction:
    def __init__(self, conn):
        self.conn = conn
        self.xid = conn._start_transaction()

    def commit(self):
        self.conn._commit_transaction(self.xid)

    def rollback(self):
        self.conn._rollback_transaction(self.xid)

The above queries the `Connection` for a new transaction-id and then calls into the `Connection` to commit or rollback transactions.  Below is how someone may use these classes without context-managers:

In [3]:
conn = Connection()
xact = Transaction(conn)

starting transaction 0


In [4]:
xact.commit()

committing transaction 0


Below is how it may look in a function:

In [5]:
def some_important_operation(conn):
    xact = Transaction(conn)
    # Example of critical stuff:
    # Creating lots of business-critical, high-value stuff here.
    # These lines will probably open whole new markets.
    # Also deliver hitherto unseen levels of shareholder value


The above example is flawed because the transaction is never committed.

## Managing transactions with context-managers

Now design a context-manager that starts a transaction, commits it if the with-block exits normall, but rolls it back in an exception occurs:

In [6]:
import contextlib

@contextlib.contextmanager
def start_transaction(connection):
    tx = Transaction(connection)

    try:
        yield tx
    except:
        tx.rollback()
        raise

    tx.commit()

The beginning of the context-manager starts a transaction.  Then, inside a try-block, there is a yield of the transaction so that transactional operations can take place inside a with-block.  Then if the with-block raises an exception, it is caught, the transaction is rolled back, and then the exception is re-raised.  If the with-block exits normally the transaction is committed and normal operations continue.

Example in action:

In [7]:
conn = Connection()
try:
    with start_transaction(conn) as tx:
        x = 1 + 1
        raise ValueError()
        y = x + 2
        print('transaction 0=', x, y)
except ValueError:
    print('Oops! Transaction 0 failed.')

starting transaction 0
rolling back transaction 0
Oops! Transaction 0 failed.


In the above, the conext manager detected the exception and rolled the transaction back.

One more time, this time without the exception:

In [8]:
try:
    with start_transaction(conn) as tx:
        x = 1 + 1
        y = x + 2
        print('transaction 1 = ', x, y)
except ValueError:
    assert False

starting transaction 1
transaction 1 =  2 4
committing transaction 1


The above is now a full-featured context manager that does pretty realistic work.  All it needs is a full database...